In [1]:
#First we upload our packages
import arcpy
from arcpy import env  
from arcpy.sa import *
import pandas as pd
import geopandas as gpd
import numpy as np
import random
import psycopg2
from shapely.geometry import Point, Polygon

In [3]:
#Next is to read in the CSV of our September 2023 data as dataframe
csv_file_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\temp_data.csv"
temp_data = pd.read_csv(csv_file_path)

In [4]:
#Now let's remove any cells with NA values
temp_data.dropna(inplace=True)

In [5]:
# Then we must convert numeric columns to appropriate data types
numeric_cols = ['high_F', 'low_F', 'precip', 'snow_inch', 'snowd_inch']
temp_data[numeric_cols] = temp_data[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [6]:
#This data also needs coordinate info for each point
#We will use a Mesonet URL to gather reporting station location
url = "https://mesonet.agron.iastate.edu/sites/networks.php?network=MN_COOP&format=csv&nohtml=on"

# Read the CSV data from the URL into a data frame
loc_df = pd.read_csv(url)

In [7]:
# Merge the "lat" and "lon" columns from df based on the matching "nwsli" and "stid" columns
temp_data = pd.merge(temp_data, loc_df[['stid', 'lat', 'lon']], how='left', left_on='nwsli', right_on='stid')

# Drop the redundant "stid" column
temp_data.drop(columns=['stid'], inplace=True)

# Display the updated precip_data data frame
print(temp_data)

      nwsli        date      time  ...  snowd_inch      lat      lon
0     NHPM5  2023-09-01  11:00 PM  ...         0.0  45.0100 -93.3792
1     LMBM5  2023-09-01   7:00 AM  ...         0.0  44.2400 -95.3200
2     LCHM5  2023-09-01   8:00 AM  ...         0.0  45.1279 -94.5348
3     PKGM5  2023-09-01   8:00 AM  ...         0.0  47.2500 -93.5900
4     CLDM5  2023-09-01   6:00 AM  ...         0.0  43.6309 -91.5027
...     ...         ...       ...  ...         ...      ...      ...
3480  KABM5  2023-09-30   7:00 AM  ...         0.0  46.4456 -93.0283
3481  CLDM5  2023-09-30   6:00 AM  ...         0.0  43.6309 -91.5027
3482  NHPM5  2023-09-30  11:00 PM  ...         0.0  45.0100 -93.3792
3483  PKGM5  2023-09-30   8:00 AM  ...         0.0  47.2500 -93.5900
3484  LMBM5  2023-09-30   7:00 AM  ...         0.0  44.2400 -95.3200

[3485 rows x 10 columns]


In [8]:
# Define Minnesota boundary box
minnesota_boundary = Polygon([( -97.5, 43.0), (-89.0, 43.0), (-89.0, 49.5), (-97.5, 49.5)])

# Check if the data falls within the Minnesota boundary
temp_data['Coordinates'] = list(zip(temp_data.lon, temp_data.lat))
temp_data['Coordinates'] = temp_data['Coordinates'].apply(Point)
gdf = gpd.GeoDataFrame(temp_data, geometry='Coordinates')

within_minnesota = gdf[gdf.geometry.within(minnesota_boundary)]
if len(within_minnesota) == 0:
    print("No data falls within Minnesota boundary.")
else:
    print("Data falls within Minnesota boundary.")

Data falls within Minnesota boundary.


In [9]:
# Convert the "Date" column to datetime type
temp_data['date'] = pd.to_datetime(temp_data['date'])

# Check the type of the "Date" column after conversion
date_column_type = temp_data['date'].dtype

print("Type of 'date' column after conversion:", date_column_type)

Type of 'date' column after conversion: datetime64[ns]


In [10]:
#Now we need to choose the date which we will interpolate for
input_date = input("Enter a date (YYYY-MM-DD format): ")

# Convert input string to datetime object
input_date = pd.to_datetime(input_date)

# Filter DataFrame based on the input date
temp_data = temp_data[temp_data['date'] == input_date]

# Display the filtered DataFrame
print(temp_data)

Enter a date (YYYY-MM-DD format): 2023-09-09
      nwsli       date     time  ...      lat      lon               Coordinates
935   RSMM5 2023-09-09  8:00 AM  ...  44.7178 -93.0975  POINT (-93.0975 44.7178)
936   MIAM5 2023-09-09  6:00 PM  ...  45.1219 -95.9269  POINT (-95.9269 45.1219)
937   GNFM5 2023-09-09  8:00 AM  ...  48.1667 -90.8875  POINT (-90.8875 48.1667)
938   MABM5 2023-09-09  7:00 AM  ...  43.5241 -91.7616  POINT (-91.7616 43.5241)
939   HSTM5 2023-09-09  5:00 AM  ...  44.7597 -92.8689  POINT (-92.8689 44.7597)
...     ...        ...      ...  ...      ...      ...                       ...
1045  GLDM5 2023-09-09  9:00 AM  ...  44.5565 -94.2207  POINT (-94.2207 44.5565)
1046  WASM5 2023-09-09  8:00 AM  ...  44.0707 -93.5264  POINT (-93.5264 44.0707)
1047  TOWM5 2023-09-09  7:00 AM  ...  47.7553 -92.2858  POINT (-92.2858 47.7553)
1048  CSLM5 2023-09-09  8:00 AM  ...  47.3847 -94.6147  POINT (-94.6147 47.3847)
1049  AGAM5 2023-09-09  8:00 AM  ...  48.3000 -95.9833     POINT

In [11]:
# This cell will calculate average temperature for each row
temp_data['avg_temp'] = (temp_data['high_F'] + temp_data['low_F']) / 2

In [12]:
# Then these next cells will calculate evapotranspiration
# First it defines the constant for the Hargreaves method
constant = 0.0023

# Convert 'date' column to datetime object
temp_data['date'] = pd.to_datetime(temp_data['date'])

In [13]:
# Then this cell calculates ET using the Hargreaves method for each row
ET0_values = []
for index, row in temp_data.iterrows():
    avg_temperature = row['avg_temp']
    temperature_range = row['high_F'] - row['low_F']
    
    # Calculate the day of the year
    day_of_year = row['date'].dayofyear
    
    # Calculate ET0 using the Hargreaves method
    ET0 = constant * (avg_temperature + 17.8) * (temperature_range ** 0.5) * (1.0 + 0.033 * np.sin(np.deg2rad(360 * (day_of_year - 81) / 365)))
    
    ET0_values.append(ET0)

# Add the calculated ET0 values as a new column to the DataFrame
temp_data['ET'] = ET0_values

# Display the updated DataFrame
print(temp_data)


      nwsli       date     time  ...               Coordinates  avg_temp        ET
935   RSMM5 2023-09-09  8:00 AM  ...  POINT (-93.0975 44.7178)      63.0  1.024494
936   MIAM5 2023-09-09  6:00 PM  ...  POINT (-95.9269 45.1219)      70.5  1.100771
937   GNFM5 2023-09-09  8:00 AM  ...  POINT (-90.8875 48.1667)      57.5  0.718715
938   MABM5 2023-09-09  7:00 AM  ...  POINT (-91.7616 43.5241)      56.5  0.988060
939   HSTM5 2023-09-09  5:00 AM  ...  POINT (-92.8689 44.7597)      64.0  0.927676
...     ...        ...      ...  ...                       ...       ...       ...
1045  GLDM5 2023-09-09  9:00 AM  ...  POINT (-94.2207 44.5565)      67.5  0.947001
1046  WASM5 2023-09-09  8:00 AM  ...  POINT (-93.5264 44.0707)      64.0  1.002005
1047  TOWM5 2023-09-09  7:00 AM  ...  POINT (-92.2858 47.7553)      59.5  0.858185
1048  CSLM5 2023-09-09  8:00 AM  ...  POINT (-94.6147 47.3847)      64.0  0.846848
1049  AGAM5 2023-09-09  8:00 AM  ...     POINT (-95.9833 48.3)      61.5  0.800179

[11

In [16]:
# Then we need to create a feature class for our points
output_fc = 'Points'

# Create a new feature class
arcpy.management.CreateFeatureclass(
    arcpy.env.workspace,
    output_fc,
    'POINT',
    spatial_reference=arcpy.SpatialReference(4326)  # WGS84 Geographic Coordinate System
)

# Check the data type of the 'ET' column in the DataFrame
et_dtype = temp_data['ET'].dtype

# Add field to store ET data, ensuring correct data type
if et_dtype == 'float64':
    arcpy.management.AddField(output_fc, 'ET', 'FLOAT')
else:
    arcpy.management.AddField(output_fc, 'ET', 'DOUBLE')

# Add 'date' and 'nwsli' fields
arcpy.management.AddField(output_fc, 'date', 'DATE')
arcpy.management.AddField(output_fc, 'nwsli', 'TEXT')



# Open an insert cursor
with arcpy.da.InsertCursor(output_fc, ['SHAPE@XY', 'ET', 'date', 'nwsli']) as cursor:
    # Iterate over each row in the DataFrame
    for index, row in temp_data.iterrows():
        # Extract lat, lon, ET, date, and nwsli values
        lat = row['lat']
        lon = row['lon']
        date = row['date']
        ET = row['ET']
        nwsli = row['nwsli']
        
        # Create a point geometry
        point = arcpy.Point(lon, lat)
        point_geometry = arcpy.PointGeometry(point)
        
        # Insert the point feature with the ET, date, and nwsli values
        cursor.insertRow([point_geometry, ET, date, nwsli])

print(f"Feature class '{output_fc}' created successfully.")


Feature class 'Points' created successfully.


In [70]:
# In order to test our interpolations later, we will need to create a subset of this data and remove it from the feature layer
input_feature_layer = "Points"

# We will save it as a separate feature layer
output_feature_layer = "Random_Selected_Points"
output_feature_class = "Random_Selected_Points"

# We first need to get the total count of features in the input feature layer
total_features_count = int(arcpy.GetCount_management(input_feature_layer).getOutput(0))

# The I will generate a list of 32 random indices
random_indices = random.sample(range(1, total_features_count + 1), 32)

# I will use a SQL expression to select the randomly chosen features
sql_expression = "OBJECTID IN ({})".format(','.join(map(str, random_indices)))

# Then I will create a new feature layer with the randomly selected features
arcpy.MakeFeatureLayer_management(input_feature_layer, output_feature_layer, sql_expression)

print("Randomly selected 32 features and created a new feature layer:", output_feature_layer)

# The I will save the selected features to a new feature class
arcpy.CopyFeatures_management(output_feature_layer, output_feature_class)
print("Saved the selected features to a new feature class:", output_feature_class)

Randomly selected 32 features and created a new feature layer: Random_Selected_Points
Saved the selected features to a new feature class: Random_Selected_Points


In [17]:
#This performs the IDW interpolation
outIDW = Idw("Points.shp", "ET")
output_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\IDW_ETPoints.tif"
outIDW.save(output_path)

In [18]:
#This performs the ordinary kriging interpolation
outKriging = Kriging("Points.shp", "ET", KrigingModelOrdinary("SPHERICAL", 0.021507), 0.0215068000000001)
output_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\Kriging_ETPoints.tif"
outKriging.save(output_path)

In [19]:
#This performs the universal kriging interpolation
outKriging = Kriging("Points.shp", "ET", KrigingModelUniversal("SPHERICAL", 0.021507), 0.0215068000000001)
output_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\Univ_Kriging_ETPoints.tif"
outKriging.save(output_path)

In [80]:
#To test the model, we need to sample the data using the randomly select points
arcpy.sa.Sample(
    in_rasters="Idw_ETPoints.tif",
    in_location_data="Random_Selected_Points",
    out_table=r"Sample_Idw_ET",
    resampling_type="NEAREST",
    unique_id_field="OBJECTID",
    process_as_multidimensional="CURRENT_SLICE",
    acquisition_definition=None,
    statistics_type="",
    percentile_value=None,
    buffer_distance=None,
    layout="ROW_WISE",
    generate_feature_class="TABLE"
)
print("Sampling complete")

Sampling complete


In [72]:
arcpy.sa.Sample(
    in_rasters="Kriging_ETPoints.tif",
    in_location_data="Random_Selected_Points",
    out_table=r"Sample_Kriging_ET",
    resampling_type="NEAREST",
    unique_id_field="OBJECTID",
    process_as_multidimensional="CURRENT_SLICE",
    acquisition_definition=None,
    statistics_type="",
    percentile_value=None,
    buffer_distance=None,
    layout="ROW_WISE",
    generate_feature_class="TABLE"
)
print("Sampling complete")

Sampling complete


In [74]:
arcpy.sa.Sample(
    in_rasters="Univ_Kriging_ETPoints.tif",
    in_location_data="Random_Selected_Points",
    out_table=r"Univ_Sample_Kriging_ET",
    resampling_type="NEAREST",
    unique_id_field="OBJECTID",
    process_as_multidimensional="CURRENT_SLICE",
    acquisition_definition=None,
    statistics_type="",
    percentile_value=None,
    buffer_distance=None,
    layout="ROW_WISE",
    generate_feature_class="TABLE"
)
print("Sampling complete")

Sampling complete


In [75]:
# Next we will create a dataframe based on our randomly selected features from earlier
input_feature_class = "Random_Selected_Points"

# We will first convert feature classes to a NumPy array
fields = ["OID@","nwsli", "ET"]
array = arcpy.da.FeatureClassToNumPyArray(input_feature_class, fields)

# Then convert the NumPy array to a dataframe
df_ET = pd.DataFrame(array)

# Rename the OID@ field to ObjectID
df_ET.rename(columns={"OID@": "OBJECTID"}, inplace=True)

# Then we will print the DataFrame to check it
print(df_ET)


    OBJECTID  nwsli        ET
0          1  MIAM5  1.100771
1          2  HSTM5  0.927676
2          3  WAAM5  0.977935
3          4  WHTM5  1.062532
4          5  MRAM5  0.953007
5          6  SPTM5  1.013507
6          7  KIMM5  0.965256
7          8  PTNM5  0.996317
8          9  MLNM5  1.100771
9         10  ELKM5  0.784732
10        11  WRIM5  0.847083
11        12  TCYM5  1.033615
12        13  ITAM5  0.977935
13        14  FORM5  1.022095
14        15  ADVM5  1.018807
15        16  ASNM5  1.024494
16        17  MMLM5  1.209295
17        18  BAUM5  0.941848
18        19  DTRM5  0.944762
19        20  TOHM5  0.656554
20        21  PKGM5  0.830632
21        22  CLDM5  0.894719
22        23  ARTM5  0.883674
23        24  SBNM5  0.924797
24        25  INDM5  0.691201
25        26  NYMM5  1.037173
26        27  MLCM5  0.965256
27        28  EMBM5  0.882313
28        29  WNNM5  0.989164
29        30  WLDM5  1.005712
30        31  GLDM5  0.947001
31        32  WASM5  1.002005


In [76]:
# Now I will add the sampled interpolated data to the same dataframe
# That will be done by first making individual dataframes for each method
table_paths = [
    "Sample_Idw_ET",
    "Sample_Kriging_ET",
    "Univ_Sample_Kriging_ET"
]

dfs = []
for table_path in table_paths:
    array = arcpy.da.TableToNumPyArray(table_path, "*")
    df = pd.DataFrame(array)
    dfs.append(df)

# To check, let's print the dataframes
for i, df in enumerate(dfs):
    print(f"DataFrame {i+1} ({table_paths[i]}):\n{df}\n")


DataFrame 1 (Sample_Idw_ET):
    OBJECTID  Random_Selected_Points         X         Y  IDW_ETPoints_Band_1
0          1                       1 -95.92690  45.12190             1.100795
1          2                       2 -92.86890  44.75970             0.927730
2          3                       3 -96.77590  48.19530             0.977890
3          4                       4 -96.48330  45.80000             1.062531
4          5                       5 -93.30720  45.92000             0.952905
5          6                       6 -93.99000  44.31000             1.013430
6          7                       7 -94.30560  45.35330             0.964862
7          8                       8 -92.07470  43.67250             0.996261
8          9                       9 -95.93000  45.12000             1.100795
9         10                      10 -93.58420  45.30490             0.784877
10        11                      11 -93.06670  46.71670             0.847116
11        12                      1

In [77]:
# Thne we iterate over each data frame in dfs list and merge with df_original
for df in dfs:
    # Perform the merge based on the OBJECTID column
    df_ET = pd.merge(df_ET, df, on='OBJECTID', how='left')

# Check the result
print(df_ET)

    OBJECTID  nwsli        ET  ...         X         Y  Univ_Kriging_ETPoints_Band_1
0          1  MIAM5  1.100771  ... -95.92690  45.12190                      1.117587
1          2  HSTM5  0.927676  ... -92.86890  44.75970                      0.932135
2          3  WAAM5  0.977935  ... -96.77590  48.19530                      0.940173
3          4  WHTM5  1.062532  ... -96.48330  45.80000                      1.041204
4          5  MRAM5  0.953007  ... -93.30720  45.92000                      0.925839
5          6  SPTM5  1.013507  ... -93.99000  44.31000                      1.003034
6          7  KIMM5  0.965256  ... -94.30560  45.35330                      0.927090
7          8  PTNM5  0.996317  ... -92.07470  43.67250                      0.973996
8          9  MLNM5  1.100771  ... -95.93000  45.12000                      1.117587
9         10  ELKM5  0.784732  ... -93.58420  45.30490                      0.850771
10        11  WRIM5  0.847083  ... -93.06670  46.71670           

In [79]:
# Let's calculate RMSE, MAE, and R-squared for each method

# Extract relevant columns
observed = df_ET['ET']
idw_predicted = df_ET['IDW_ETPoints_Band_1']
kriging_predicted = df_ET['Kriging_ETPoints_Band_1']
univ_kriging_predicted = df_ET['Univ_Kriging_ETPoints_Band_1']

# Calculate RMSE, MAE, and R-squared for IDW
idw_rmse = np.sqrt(((observed - idw_predicted) ** 2).mean())
idw_mae = np.abs(observed - idw_predicted).mean()
idw_r2 = 1 - (((observed - idw_predicted) ** 2).sum() / ((observed - observed.mean()) ** 2).sum())

# Calculate RMSE, MAE, and R-squared for Kriging
kriging_rmse = np.sqrt(((observed - kriging_predicted) ** 2).mean())
kriging_mae = np.abs(observed - kriging_predicted).mean()
kriging_r2 = 1 - (((observed - kriging_predicted) ** 2).sum() / ((observed - observed.mean()) ** 2).sum())

# Calculate RMSE, MAE, and R-squared for Universal Kriging
univ_kriging_rmse = np.sqrt(((observed - univ_kriging_predicted) ** 2).mean())
univ_kriging_mae = np.abs(observed - univ_kriging_predicted).mean()
univ_kriging_r2 = 1 - (((observed - univ_kriging_predicted) ** 2).sum() / ((observed - observed.mean()) ** 2).sum())

# Print the results
print("IDW:")
print("RMSE:", idw_rmse)
print("MAE:", idw_mae)
print("R-squared:", idw_r2)
print("------------------------")
print("Kriging:")
print("RMSE:", kriging_rmse)
print("MAE:", kriging_mae)
print("R-squared:", kriging_r2)
print("------------------------")
print("Universal Kriging:")
print("RMSE:", univ_kriging_rmse)
print("MAE:", univ_kriging_mae)
print("R-squared:", univ_kriging_r2)


IDW:
RMSE: 0.00058134634
MAE: 0.00028793328
R-squared: 0.9999721332242189
------------------------
Kriging:
RMSE: 0.059642993
MAE: 0.04458194
R-squared: 0.706684023141861
------------------------
Universal Kriging:
RMSE: 0.059642993
MAE: 0.04458194
R-squared: 0.706684023141861


In [ ]:
# Exploratory Interpolation
arcpy.ga.ExploratoryInterpolation(
    in_features="Points",
    value_field="ET",
    out_cv_table=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb\ExploratoryInterpolation1",
    out_geostat_layer=None,
    interp_methods="ORDINARY_KRIGING;UNIVERSAL_KRIGING;IDW",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)


In [ ]:
#This performs the resampling of our chosen tif to reduce the number of points needed
arcpy.management.Resample(
    in_raster=r"Idw_ETPoints.tif",
    out_raster=r"Idw_ET_Resample",
    cell_size="0.2 0.2",
    resampling_type="NEAREST"
)

In [21]:
#This converts the raster to points
arcpy.conversion.RasterToPoint(
    in_raster=r"Idw_ET_Resample",
    out_point_features=r"Idw_ET_Point",
    raster_field="value"
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5572 Final\\GIS 5572 Final.gdb\\Univ_Kriging_ET_Point'>

In [25]:
#This will export the IDW points to a PostGIS database
arcpy.conversion.ExportFeatures(
    in_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb\Idw_ET_Point",
    out_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\PostgreSQL-34-final_project(postgres).sde\final_project.postgres.et_data"
)
print("Export complete")

Export complete
